# Code for Calculating the Frobenius Norm of target unitary from different GRAPE functions

In this notebook, we use two different functions from the qutip control library ("optimize_pulse_unitary" from the pulseoptim object and "cy_grape_unitary" from the grape object), in order to implement the GRAPE algorithm. From this we obtain a final Unitary $\hat U$ which should be close to some given target Unitary $\hat U^*$. We then calculate the Frobenius norm squared of the difference between these two unitary as a measure of the error fidelity:

$$ || A  ||_{F}^2  = Tr(A^{\dagger}A) = \sum_{ij}^{N} |A_{ij}|^{2}$$


These values are then plotted against previously obtained results which obtained the unitary by reformulating the system in terms of a polynomial equation (https://arxiv.org/pdf/2209.05790.pdf) 

## Imports

Below are the necessary imports for running this code:

1) Qutip functions: 


The first line does import all qutip functions from the base object, which is used here to convery numpy arrays to Quantum objects using Qobj() and qeye() to create an identity operator of a specified size (here 3) as the initial starting point for the GRAPE algorithm.

The next set of imports are functions used for the pulseoptim class and a logger class which formats the result when being printed to terminal/ saving the to a file (I don't think we need them)

The final set of qutip imports are for implementing the cy_grape method. The TextProgressBar is justed used as a convenient way of tracking the progress of the function. (plot_grape_control_fields and _overlap are not necessary any more but are just used as tools for plotting the grape control fields and calculating the trace norm respectively)

2) Other imports:


The next set of imports are just the basic imports for simple tasks: 
matplotlib for plotting. 

numpy for storing and manipulating data.

h5py for reading and writing hdf5 files. 

time for time-keeping.  


In [1]:
from qutip import *

#QuTiP control modules
import qutip.control.pulseoptim as cpo
#import qutip.logging_utils as logging
#logger = logging.get_logger()
#Set this to None or logging.WARN for 'quiet' execution
#log_level = logging.INFO

from qutip.control import * 
from qutip.ui.progressbar import TextProgressBar
from qutip.control.grape import plot_grape_control_fields, _overlap


%matplotlib inline
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
#import datetime
import h5py
import time 
start_time = time.time()

from tqdm.notebook import tqdm

from multiprocessing import Pool

In [2]:
import numpy as np
from functools import reduce
    
# Identity and Pauli matrices
I2 = np.eye(2)
X = np.array([[0.0, 1.0], [1.0, 0.0]])
Z = np.array([[1.0, 0.0], [0.0, -1.0]])
    
# Kronecker product of a single operator at position `pos` in n-qubit system
def kron_n_single(n, op, pos):
    ops = [I2.copy() for _ in range(n)]
    ops[pos] = op
    return reduce(np.kron, ops)
    
# Kronecker product of two operators at positions `pos1` and `pos2`
def kron_n_double(n, op1, pos1, op2, pos2):
    ops = [I2.copy() for _ in range(n)]
    ops[pos1] = op1
    ops[pos2] = op2
    return reduce(np.kron, ops)
    
# Static (ZZ) part of the Ising Hamiltonian
def H0_Ising(n):
    dim = 2**n
    H0 = np.zeros((dim, dim))
    for i in range(n - 1):
        H0 += kron_n_double(n, Z, i, Z, i + 1)
    return H0
    
# Control (X) part of the Ising Hamiltonian
def Hc_Ising(n):
    dim = 2**n
    Hc = np.zeros((dim, dim))
    for i in range(n):
        Hc += kron_n_single(n, X, i)
    return Hc

In [3]:
# Fidelity error target
fid_err_targ = 1e-10
# Maximum iterations for the optisation algorithm
max_iter = 1000
# Maximum (elapsed) time allowed in seconds
max_wall_time = 1000
# Minimum gradient (sum of gradients squared)
# as this tends to 0 -> local minima has been found
min_grad = 1e-10
    
# pulse type alternatives: RND|ZERO|LIN|SINE|SQUARE|SAW|TRIANGLE|
p_type = 'RND' 

In [4]:
def optimize(U_tag, H0, Hc, U_0, alg):
    return cpo.optimize_pulse_unitary(
                H0, Hc, U_0, Qobj(U_tag), n_ts, evo_time, 
                fid_err_targ=fid_err_targ, min_grad=min_grad, 
                max_iter=max_iter, max_wall_time=max_wall_time, 
                # log_level=log_level,
                init_pulse_type=p_type, 
                alg=alg,
                # gen_stats=True
            ).evo_full_final

In [5]:
for j in [2,3,4,5]:

    n_ts = 10

    with h5py.File(f"results_{j}_100.hdf5", 'r') as resultsFile:
        #extract the data and store in an array
    
        U_targets = resultsFile["U_targets"][...]
        
    sample = U_targets.shape[2] #choose number of target unitaries - this is choosing the full 1000
    
    #for storing the obtained unitaries (*)
    U_final_pulseoptim = []
    U_final_cyGRAPE = []
    
    
    H0 = H0_Ising(j)
    H0 /= H0.max()
    
    Hc = Hc_Ising(j)
    Hc /= Hc.max()
    
    H0 = Qobj(H0)
    Hc = [Qobj(Hc)] 
     
    # Unitary starting point
    U_0 = qeye(2**j)
    
    
    # Number of time slots
    
    
    # Time allowed for the evolution
    evo_time = 0.5

    U_final_pulseoptim = [
        optimize(U_targets[:,:,i], H0, Hc, U_0, alg='CRAB') for i in tqdm(range(sample))
    ]

    U_final_cyGRAPE = [
        optimize(U_targets[:,:,i], H0, Hc, U_0, alg='GRAPE') for i in tqdm(range(sample))
    ]

    #calculating fidelity
    
    pulseoptim_fidelity = np.zeros(sample)
    grape_fidelity = np.zeros(sample)
    
    
    for i in range(sample):
        pulseoptim_fidelity[i] = abs(_overlap(Qobj(U_targets[:,:,i]), U_final_pulseoptim[i]))
        grape_fidelity[i] = abs(_overlap(Qobj(U_targets[:,:,i]), U_final_cyGRAPE[i]))
        
    
    
    pulseoptim_inf = 1 - pulseoptim_fidelity 
    grape_inf = 1 - grape_fidelity

    with h5py.File('RND_qutip_opt_results_qubits_' +str(j) + '_100.hdf5', 'w') as hf:
        hf.create_dataset("pulseoptim_inf",  data=pulseoptim_inf)
        hf.create_dataset("grape_inf",  data= grape_inf)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]